In [7]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "san-francisco_california.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Plaza"

            "Trail", "Parkway", "Commons", "Circle", "Crescent", "Gate", "Terrace", "Grove", "Way"]
#expected names in the dataset




mapping = {  "St": "Street",

            "St.": "Street",
           
            "st": "Street",

            "STREET": "Street",

            "Ave": "Avenue",

            "Ave.": "Avenue",

            "Dr.": "Drive",

            "Dr": "Drive",

            "Rd": "Road",

            "Rd.": "Road",

            "Blvd": "Boulevard",

            "Blvd.": "Boulevard",

            "Ehs": "EHS",

            "Trl": "Trail",

            "Cir": "Circle",

            "Cir.": "Circle",

            "Ct": "Court",

            "Ct.": "Court",

            "Crt": "Court",

            "Crt.": "Court",

            "By-pass": "Bypass",
           
            "Plz":"Plaza",

            "N.": "North",

            "N": "North",

            "E.": "East",

            "E": "East",

            "S.": "South",
           
            "Socity": "Society",           

            "S": "South",

            "W.": "West",

            "W": "West"

          }
# Search string for the regex. If it is matched and not in the expected list then add this as a key to the set.
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):# Check if it is a street name
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):# return the list that satify the above two functions
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


# return the updated names
def update_name(name, mapping):
    name1=name.split()
    for x in name1:
        if x in mapping:
            name1.remove(x)
            name1.append(mapping[x])
            name=' '.join(name1)
    return name

st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name,"=>", better_name

{'1': set(['W Of Us 101 @ Jct Sr 1']),
 '100': set(['Woodside Road, Suite 100']),
 '105': set(['Grand Avenue #105', 'N California Blvd #105']),
 '10675': set(['10675']),
 '12180142': set(['12180142']),
 u'122\xb029\'07.1"W': set([u'37\xb042\'46.0"N 122\xb029\'07.1"W']),
 '15': set(['Doolittle Dr, Suite 15']),
 '151': set(['Brannan St #151']),
 '155': set(['University Ave #155', 'Woodside Road, Suite 155']),
 '15th': set(['15th']),
 '2': set(['San Francisco Bicycle Route 2']),
 '203': set(['Bartlett Street #203']),
 '3': set(['University Dr. Suite 3']),
 '3.2': set(['ALA 84 PM 3.2']),
 '302': set(['Mission Blvd #302']),
 '3500': set(['Sansome St #3500']),
 '3658': set(['Market Street Suite 3658']),
 '39': set(['Pier 39']),
 '4': set(['Menlo Ave  # 4']),
 '4.5': set(['SF 80 PM 4.5']),
 '404': set(['16th St #404']),
 '41276': set(['Upton St 41276']),
 '730': set(['Sansome Street Ste 730', 'Sansome Street Suite 730']),
 'A': set(['Adrian Court, Suite A', 'Pier 50 A', 'Upton St #A']),
 'AVE